In [ ]:
import cv2
from deepface import DeepFace
import os
from tqdm import tqdm

from insightface.app import FaceAnalysis
import uuid 

def detect_emotions(video_path, output_path):
    # Capturar vídeo do arquivo especificado
    cap = cv2.VideoCapture(video_path)

    # Verificar se o vídeo foi aberto corretamente
    if not cap.isOpened():
        print("Erro ao abrir o vídeo.")
        return

    # Obter propriedades do vídeo
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Definir o codec e criar o objeto VideoWriter
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec para MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    app = FaceAnalysis()
    app.prepare(ctx_id=0)  # Use ctx_id=-1 for CPU
    
    diretorio_saida="detected_faces"
    
    criar_pasta_para_rostos(diretorio=diretorio_saida)
    
   
    Rastreador = {}
    proximo_id = 1
    Rastreados = set()

    # Loop para processar cada frame do vídeo
    for frame_idx in tqdm(range(total_frames), desc="Processando vídeo"):
        # Ler um frame do vídeo
        ret, frame = cap.read()

        # Se não conseguiu ler o frame (final do vídeo), sair do loop
        if not ret:
            break

        rgb_frame = frame[:, :, ::-1]

        faces = app.get(rgb_frame)

        for face in faces:
            box = face.bbox.astype(int)
            
            x1, y1, x2, y2=ajustar_limites_rosto(box, width, height)

            cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (255, 0, 0), 2)

            face_roi = rgb_frame[y1:y2, x1:x2]
            
            ja_existe = False
            id_face = None
            
            if face_roi.size == 0:
                print(f"Rosto inválido detectado no frame {frame_idx}, ignorando.")
                continue
            else:
                face_id, proximo_id, Rastreador = rastrear_ou_criar_id(box, Rastreador, proximo_id)
                Rastreador[face_id] = ((x1, y1, x2, y2), frame_idx)
                salvar_rosto(face_roi, face_id, Rastreados, diretorio_saida)
                     



            try:

                analysis = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)

                if isinstance(analysis, list) and len(analysis) > 0:
                    dominant_emotion = analysis[0]['dominant_emotion']
                else:
                    dominant_emotion = "Unknown"

            except Exception as e:
                dominant_emotion = "Error"
                print(f"Error analyzing emotion: {e}")

            cv2.putText(frame, dominant_emotion, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        # Escrever o frame processado no arquivo de vídeo de saída   
        out.write(frame)

    # Liberar a captura de vídeo e fechar todas as janelas
    cap.release()
    out.release()
    cv2.destroyAllWindows()




def criar_pasta_para_rostos(diretorio="detected_faces"):
    # Obter o caminho absoluto
    caminho_absoluto = os.path.abspath(diretorio)
    
    # Criar a pasta se ela não existir
    if not os.path.exists(caminho_absoluto):
        os.makedirs(caminho_absoluto)
        print(f"Pasta criada: {caminho_absoluto}")
    else:
        print(f"Pasta já existente: {caminho_absoluto}")
        return caminho_absoluto

# Exemplo de uso
pasta_rostos = criar_pasta_para_rostos()

def ajustar_limites_rosto(box, width, height):
   
    x1, y1, x2, y2 = box
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(width, x2), min(height, y2)
    return x1, y1, x2, y2
def rastrear_ou_criar_id(box, face_trackers, proximo_id):
  
    for existing_id, tracker_info in face_trackers.items():
        tracked_box, _ = tracker_info
        iou = sobreposicao(box, tracked_box)
        if iou > 0.5:
            return existing_id, proximo_id, face_trackers
    
    face_id = proximo_id
    proximo_id += 1
    return face_id, proximo_id, face_trackers
#face_roi, face_id, Rastreados, diretorio_saida

def salvar_rosto(face_roi, face_id, Rastreados, diretorio_saida):

    if face_id not in Rastreados:
        Rastreados.add(face_id)
        face_image_path = os.path.join(diretorio_saida, f"face_{face_id}.jpg")
        cv2.imwrite(face_image_path, cv2.cvtColor(face_roi, cv2.COLOR_RGB2BGR))

def sobreposicao(boxA, boxB):
    # Calcular a interseção sobre união (IOU) entre dois retângulos
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    iou = interArea / float(boxAArea + boxBArea - interArea)

    return iou


# Caminho para o arquivo de vídeo na mesma pasta do script
script_dir = os.getcwd()
input_video_path = os.path.join(script_dir, 'Video_tc.mp4')
output_video_path = os.path.join(script_dir, 'output_video_11.mp4')  # Nome do vídeo de saída





# Chamar a função para detectar emoções no vídeo e salvar o vídeo processado
detect_emotions(input_video_path, output_video_path)


Pasta criada: c:\Users\nando\OneDrive\Área de Trabalho\Rec.facial\TC4\tech-challenge-fase-4\detected_faces
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\nando/.insightface\models\buffalo_l\1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\nando/.insightface\models\buffalo_l\2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\nando/.insightface\models\buffalo_l\det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\nando/.insightface\models\buffalo_l\genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
fin

Processando vídeo:   7%|▋         | 225/3326 [01:52<25:56,  1.99it/s]


KeyboardInterrupt: 